# 特征工程

In [1]:
import numpy as np
import pandas as pd
from jesse import helpers, research

from custom_indicators.all_features import feature_matrix

warmup_candles, trading_candles = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "3m",
    helpers.date_to_timestamp("2021-01-01"),
    helpers.date_to_timestamp("2024-12-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

final_fe = feature_matrix(trading_candles, sequential=True)
final_fe.features.shape

(700800, 394)

In [2]:
invalid_len = (np.isfinite(final_fe.features) == False).sum(axis=0).max()
invalid_len

50

# 模型

In [3]:
invalid_len = 200

train_mask = trading_candles[:, 0] < helpers.date_to_timestamp("2024-06-30")
vali_mask = (trading_candles[:, 0] >= helpers.date_to_timestamp("2024-06-30")) & (
    trading_candles[:, 0] < helpers.date_to_timestamp("2024-09-30")
)
test_mask = trading_candles[:, 0] >= helpers.date_to_timestamp("2024-09-30")

train_len = train_mask.sum()
vali_len = vali_mask.sum()
test_len = test_mask.sum()
assert train_len + vali_len + test_len == trading_candles.shape[0]

final_fe_df = pd.DataFrame(
    final_fe.features, columns=final_fe.names, index=trading_candles[:, 0]
)
final_fe_df.head()

,acc_swing_index,acc_swing_index_lag1,acc_swing_index_lag2,acc_swing_index_dt,acc_swing_index_ddt,ehlers_early_onset_trend_ddt,bandpass,bandpass_dt,bandpass_ddt,highpass_bp,...,swamicharts_stochastic_39,swamicharts_stochastic_40,swamicharts_stochastic_41,swamicharts_stochastic_42,swamicharts_stochastic_43,swamicharts_stochastic_44,swamicharts_stochastic_45,swamicharts_stochastic_46,swamicharts_stochastic_47,swamicharts_stochastic_48
1.609459e+12,0.000000,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.609459e+12,-0.038929,0.000000,NaN,-0.038929,NaN,NaN,0.983821,-0.016179,NaN,0.852647,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.609460e+12,-0.064255,-0.038929,0.000000,-0.025326,0.013604,1.000000,0.874514,-0.109307,-0.093129,0.637810,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.609460e+12,-0.152038,-0.064255,-0.038929,-0.087783,-0.062457,-1.000000,0.689142,-0.185372,-0.076064,0.380348,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.609460e+12,-0.217581,-0.152038,-0.064255,-0.065544,0.022239,-0.228989,0.450630,-0.238512,-0.053140,0.107308,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
label_df = pd.read_csv("data/label.csv.zst", index_col=0)
label_df.head()

,close,ret,trgt,bin,side
2021-01-01 00:00:00+00:00,29016.23,NaN,NaN,0.0,0.0
2021-01-01 00:03:00+00:00,28965.02,NaN,NaN,0.0,0.0
2021-01-01 00:06:00+00:00,28928.06,NaN,NaN,0.0,0.0
2021-01-01 00:09:00+00:00,28851.82,NaN,NaN,0.0,0.0
2021-01-01 00:12:00+00:00,28786.75,NaN,NaN,0.0,0.0


In [5]:
train_fe = final_fe_df[train_mask].iloc[invalid_len:]
train_label = label_df[train_mask].iloc[invalid_len:]

train_fe = train_fe[train_label["ret"].notna().to_numpy()]
train_label = train_label[train_label["ret"].notna().to_numpy()]["bin"].astype(int)

assert train_fe.shape[0] == train_label.shape[0]
print(train_label.value_counts())

vali_fe = final_fe_df[vali_mask].iloc[invalid_len:]
vali_label = label_df[vali_mask].iloc[invalid_len:]

vali_fe = vali_fe[vali_label["ret"].notna().to_numpy()]
vali_label = vali_label[vali_label["ret"].notna().to_numpy()]["bin"].astype(int)

assert vali_fe.shape[0] == vali_label.shape[0]
print(vali_label.value_counts())

test_fe = final_fe_df[test_mask].iloc[invalid_len:]
test_label = label_df[test_mask].iloc[invalid_len:]

test_fe = test_fe[test_label["ret"].notna().to_numpy()]
test_label = test_label[test_label["ret"].notna().to_numpy()]["bin"].astype(int)

assert test_fe.shape[0] == test_label.shape[0]
print(test_label.value_counts())

bin
0    437622
1     32211
Name: count, dtype: int64
bin
0    31550
1     2299
Name: count, dtype: int64
bin
0    31315
1     2399
Name: count, dtype: int64


In [7]:
import optuna
import lightgbm as lgb
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    dtrain = lgb.Dataset(train_fe, label=train_label)

    # 设定需要调整的超参数范围
    param = {
        "objective": "binary",
        "boosting_type": trial.suggest_categorical("boosting_type", ["gbdt", "dart"]),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.1),
        "num_leaves": trial.suggest_int("num_leaves", 31, 512),
        "max_depth": trial.suggest_int("max_depth", -1, 16),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.5, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.5, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 0, 5),
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
        # 针对正负样本数严重不平衡的情况，可以设置scale_pos_weight
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1.0, 20.0),
        "verbosity": -1,
    }

    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(vali_fe)
    pred_labels = (preds >= trial.suggest_uniform("label_threshold", 0.5, 1.0)).astype(
        int
    )

    # 计算AUC、精确率、召回率
    auc_score = roc_auc_score(vali_label, preds, average="weighted")
    precision = precision_score(vali_label, pred_labels, zero_division=0)
    recall = recall_score(vali_label, pred_labels, zero_division=0)
    f1 = f1_score(vali_label, pred_labels, zero_division=0)
    # 输出精确率与召回率供参考
    print(
        f"AUC: {auc_score:.5f}, Precision: {precision:.5f}, Recall: {recall:.5f}, F1: {f1:.5f}"
    )

    return f1


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=500, show_progress_bar=True)

[I 2025-02-10 23:31:12,913] A new study created in memory with name: no-name-cadcd388-2e07-447f-9e3d-c30df33de88a
Best trial: 0. Best value: 0:   0%|          | 1/500 [00:11<1:35:01, 11.43s/it]

AUC: 0.67709, Precision: 0.00000, Recall: 0.00000, F1: 0.00000
[I 2025-02-10 23:31:24,395] Trial 0 finished with value: 0.0 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.01162869786456743, 'num_leaves': 279, 'max_depth': 8, 'feature_fraction': 0.9198128745615626, 'bagging_fraction': 0.9114948512185802, 'bagging_freq': 5, 'lambda_l1': 1.2256021271607538, 'lambda_l2': 0.7069892785932729, 'scale_pos_weight': 5.825699005269939, 'label_threshold': 0.7205625132168395}. Best is trial 0 with value: 0.0.


Best trial: 1. Best value: 0.2144:   0%|          | 2/500 [00:19<1:16:26,  9.21s/it]

AUC: 0.67368, Precision: 0.15484, Recall: 0.34841, F1: 0.21440
[I 2025-02-10 23:31:32,053] Trial 1 finished with value: 0.21440042826552463 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.05185197073809939, 'num_leaves': 398, 'max_depth': 7, 'feature_fraction': 0.8970305822423044, 'bagging_fraction': 0.8111453724083731, 'bagging_freq': 5, 'lambda_l1': 0.10520460763681902, 'lambda_l2': 3.957691804579501e-05, 'scale_pos_weight': 15.057044781910013, 'label_threshold': 0.5866676541692166}. Best is trial 1 with value: 0.21440042826552463.


Best trial: 1. Best value: 0.2144:   1%|          | 3/500 [00:30<1:25:04, 10.27s/it]

...

AUC: 0.66933, Precision: 0.25524, Recall: 0.09004, F1: 0.13312
[I 2025-02-10 23:37:33,338] Trial 37 finished with value: 0.13311897106109324 and parameters: {'boosting_type': 'dart', 'learning_rate': 0.05690823191653845, 'num_leaves': 304, 'max_depth': 8, 'feature_fraction': 0.5056620396195834, 'bagging_fraction': 0.9715694831129693, 'bagging_freq': 2, 'lambda_l1': 1.2791903648701386, 'lambda_l2': 0.002367469315955347, 'scale_pos_weight': 13.27911081164662, 'label_threshold': 0.6371179738715037}. Best is trial 26 with value: 0.21779172148567957.


Best trial: 26. Best value: 0.217792:   8%|▊         | 39/500 [06:39<1:32:13, 12.00s/it]

AUC: 0.67397, Precision: 0.00000, Recall: 0.00000, F1: 0.00000
[I 2025-02-10 23:37:52,469] Trial 38 finished with value: 0.0 and parameters: {'boosting_type': 'dart', 'learning_rate': 0.02021564344382161, 'num_leaves': 441, 'max_depth': 11, 'feature_fraction': 0.6569739812765429, 'bagging_fraction': 0.8063236441122529, 'bagging_freq': 1, 'lambda_l1': 4.189685664281005e-07, 'lambda_l2': 1.2179089238142366e-05, 'scale_pos_weight': 11.59805715675364, 'label_threshold': 0.6881110455167117}. Best is trial 26 with value: 0.21779172148567957.


Best trial: 26. Best value: 0.217792:   8%|▊         | 40/500 [06:54<1:39:09, 12.93s/it]

AUC: 0.66943, Precision: 0.27536, Recall: 0.01653, F1: 0.03119
[I 2025-02-10 23:38:07,569] Trial 39 finished with value: 0.031185884283955682 and parameters: {'boosting_type': 'dart', 'learning_rate': 0.08143303180404182, 'num_leaves': 362, 'max_depth': 10, 'feature_fraction': 0.5541186170897009, 'bagging_fraction': 0.7648500567686234, 'bagging_freq': 2, 'lambda_l1': 0.0038285827369063407, 'lambda_l2': 1.7078558186384972e-07, 'scale_pos_weight': 9.354699931331218, 'label_threshold': 0.6589681685318786}. Best is trial 26 with value: 0.21779172148567957.


In [15]:
study.best_trial

array([ 72,  81,  78,  69,  54,  65, 110, 134,  80,  97], dtype=int32)